# **Project Pengembangan dan Pengoperasian Sistem Machine Learning: Studi Kasus Prediksi Berita Palsu**
`Nama: Shendi Teuku Maulana Efendi`<br>
`Username Dicoding: shendyeff`

## **Import Library yang dibutuhkan**

In [17]:
import pandas as pd
import os
import sys
from typing import Text
 
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

## **Data Processing Using Pandas**

### **Load Dataset**

Pada langkah pertama, kita memuat dataset `fake_or_real_news.csv` yang berisi informasi tentang berita yang ditandai sebagai `FAKE` atau `REAL`.

In [2]:
# Load data
df = pd.read_csv('./data/fake_or_real_news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### **Mengganti Nama Kolom**

Dataset asli memiliki kolom label yang kita ganti namanya menjadi is_fake untuk lebih mencerminkan bahwa kolom ini berisi informasi apakah berita tersebut palsu (1) atau asli (0):

In [3]:
df.rename(columns={'label': 'is_fake'}, inplace=True)
df['is_fake'] = df['is_fake'].replace({'FAKE': 1, 'REAL': 0})

In [4]:
df.head()

,Unnamed: 0,title,text,is_fake
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


### **Menyeimbangkan Dataset**

Untuk menghindari ketidakseimbangan antara kelas "FAKE" dan "REAL", kita akan mengambil jumlah sampel yang sama dari kedua kelas. Kita menentukan jumlah sampel dengan memilih nilai terkecil dari kelas yang ada, lalu kita mengambil sampel secara acak untuk masing-masing kelas:

In [5]:
num_samples = min(df['is_fake'].value_counts()) // 2 

In [6]:
fake_sample = df[df['is_fake'] == 1].sample(n=num_samples, random_state=42)
real_sample = df[df['is_fake'] == 0].sample(n=num_samples, random_state=42)

Setelah itu, kita gabungkan kedua sampel yang sudah diambil secara acak dan mengacak urutan barisnya:

In [7]:
balanced_df = pd.concat([fake_sample, real_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

### **Menghapus Kolom yang Tidak Diperlukan**

Beberapa kolom dalam dataset mungkin tidak diperlukan untuk pemodelan selanjutnya. Misalnya, kita menghapus kolom title yang tidak digunakan dalam model kita:

In [8]:
balanced_df.drop(['title'], axis=1, inplace=True)

Selain itu, kita mengganti nama kolom Unnamed: 0 menjadi id untuk memberikan label yang lebih jelas untuk kolom tersebut:

In [9]:
balanced_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

### **Menyimpan Dataset yang Sudah Seimbang**

Setelah dataset seimbang, kita simpan hasilnya dalam file CSV yang baru agar bisa digunakan dalam proses pelatihan dan pengujian model:

In [10]:
balanced_df.to_csv('./data/fake_or_real_news_balanced.csv', index=False)

### **Mengecek Distribusi Kelas**

Terakhir, kita periksa distribusi kelas dalam dataset yang sudah seimbang:

In [11]:
balanced_df['is_fake'].value_counts()

0    1582
1    1582
Name: is_fake, dtype: int64

In [12]:
balanced_df.head()

,id,text,is_fake
0,1035,(CNN) Donald Trump's suggestion that women who...,0
1,9167,It’s Over For Hillary After People See What Sh...,1
2,6527,Militia Fighters to Advance on Areas West of C...,1
3,7095,Photo by The U.S. Army | CC BY 2.0 \n\nHere is...,1
4,9450,We Are Change \nIssues surrounding Hillary Cli...,1


## **Set Variable** 

Mendefinisikan variabel penting untuk pipeline machine learning. PIPELINE_NAME memberi nama pipeline, sementara DATA_ROOT menunjukkan lokasi data. TRANSFORM_MODULE_FILE, TRAINER_MODULE_FILE, dan TUNER_MODULE_FILE merujuk ke modul untuk transformasi data, pelatihan model, dan tuning. Output pipeline disimpan di OUTPUT_BASE, dengan subfolder seperti serving_model_dir untuk model yang siap digunakan dan pipeline_root untuk hasil pipeline secara keseluruhan. metadata_path mencatat metadata terkait pipeline, memudahkan pengelolaan dan monitoring.

In [18]:
PIPELINE_NAME = "shendyeff-fake-news-pipeline"

# pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/fake_news_transform.py"
TRAINER_MODULE_FILE = "modules/fake_news_trainer.py"
TUNER_MODULE_FILE = "modules/fake_news_tuner.py"

# pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

Fungsi init_local_pipeline digunakan untuk menginisialisasi pipeline lokal di TFX. Fungsi ini menerima dua parameter, yaitu components yang berisi komponen-komponen pipeline dan pipeline_root yang menentukan lokasi direktori untuk output pipeline. Di dalam fungsi, pertama-tama akan dicatat informasi mengenai pipeline_root menggunakan logging.info. Kemudian, beam_args diatur untuk menjalankan pipeline menggunakan mode multi-proses dengan jumlah pekerja yang otomatis disesuaikan berdasarkan jumlah CPU yang tersedia. Setelah itu, fungsi ini mengembalikan objek pipeline.Pipeline yang berisi pengaturan-pengaturan seperti pipeline_name, pipeline_root, metadata_connection_config, dan beam_pipeline_args. Konfigurasi ini memungkinkan pipeline untuk dieksekusi secara lokal dengan pengaturan yang sudah ditentukan sebelumnya.

In [19]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available 
        # during execution time.
        "----direct_num_workers=0" 
    ]
    
    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

Blok kode ini menginisialisasi dan menjalankan pipeline TFX. Dimulai dengan mengatur tingkat logging ke INFO, lalu menginisialisasi komponen pipeline melalui init_components yang menerima beberapa argumen, termasuk modul pelatihan, transformasi, dan tuning. Setelah itu, pipeline lokal diinisialisasi dengan init_local_pipeline dan dijalankan menggunakan BeamDagRunner().run(pipeline=pipeline) untuk mengeksekusi pipeline di lingkungan lokal.

In [20]:
if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)
    
    from modules.components import init_components
    
    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        tuner_module=TUNER_MODULE_FILE,
        training_steps=5000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir,
    )
    
    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'embedding_dim', 'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'lstm_units', 'default': 16, 'conditions': [], 'values': [16, 32], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'dense_units', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'dropout_rate', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'num_layers', 'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': 'linear'}}], 'values': {'embeddi

Trial 1 Complete [00h 00m 11s]
val_binary_accuracy: 0.8287461996078491

Best val_binary_accuracy So Far: 0.8287461996078491
Total elapsed time: 00h 00m 11s
Results summary
Results in output\shendyeff-fake-news-pipeline\Tuner\.system\executor_execution\8\.temp\8\fake_news_tuner
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
embedding_dim: 64
lstm_units: 16
dense_units: 112
dropout_rate: 0.2
learning_rate: 0.001
num_layers: 3
Score: 0.8287461996078491


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'example_splits': 'null', 'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "is_fake"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "is_fake"\n      ]\n    }\n  ]\n}', 'fairness_indicator_thre

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to output\shendyeff-fake-news-pipeline\Evaluator\blessing\9.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 9 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'evaluation': [Artifact(artifact: uri: "output\\shendyeff-fake-news-pipeline\\Evaluator\\evaluation\\9"
, artifact_type: name: "ModelEvaluation"
)], 'blessing': [Artifact(artifact: uri: "output\\shendyeff-fake-news-pipeline\\Evaluator\\blessing\\9"
, artifact_type: name: "ModelBlessing"
)]}) for execution 9
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Evaluator is finished.
INFO:absl:node Pusher is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.pusher.component.Pusher"
    base_type: DEPLOY
  }
  id: "Pusher"
}
contexts {
  contexts {
    type {
   